In [1]:
import torch
import warnings

warnings.filterwarnings(action='ignore')

### Activations

In [2]:
import numpy as np

np.tanh(-2.2), np.tanh(0.1), np.tanh(2.5)

(-0.9757431300314515, 0.09966799462495582, 0.9866142981514303)

### NN Module